In [1]:
WORKING_ON_COLAB = True

if WORKING_ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    DATASET_FILE_PATH = '/content/drive/My Drive/Projects/IRBoardGameComplexity/dataset.csv'
    !pip install fastcoref==2.0.*
else:
    DATASET_FILE_PATH = 'data/dataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
nlp.add_pipe("fastcoref")

Downloading:   0%|          | 0.00/819 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/362M [00:00<?, ?B/s]

In [3]:
import logging

logger = logging.getLogger('bgg_predict')
logger.handlers.clear()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

logger.debug('test')

2022-11-07 22:02:44,545 bgg_predict  DEBUG    test
DEBUG:bgg_predict:test


In [4]:
import re
from typing import List
from dataclasses import dataclass

regex_mail = re.compile(r'\w+(?:\.\w+)*?@\w+(?:\.\w+)+')

@dataclass
class Sentence:
    content: str
    start: int
    end: int
    
    def does_include_pos(self, pos: int) -> bool:
        return self.start <= pos <= self.end

def clean_text(text: str) -> str:
    return regex_mail.sub('', text)

def get_sentences_from_text(text: str) -> List[Sentence]:
    # assert no continuous dots because of text cleared while building the dataset
    sentences = text.split('.')
    
    res = []
    char_accumulator = 0
    for sentence in sentences:
        res.append(Sentence(sentence, char_accumulator, char_accumulator + len(sentence) - 1))
        char_accumulator += len(sentence) + 1
        
    if res[-1].content == '':
        res.pop()
        
    return res

In [8]:
if WORKING_ON_COLAB:
    # from https://github.com/python/cpython/blob/main/Lib/bisect.py#L68
    def bisect_left(a, x, lo=0, hi=None, *, key=None):
        if hi is None:
            hi = len(a)
        # Note, the comparison uses "<" to match the
        # __lt__() logic in list.sort() and in heapq.
        if key is None:
            while lo < hi:
                mid = (lo + hi) // 2
                if a[mid] < x:
                    lo = mid + 1
                else:
                    hi = mid
        else:
            while lo < hi:
                mid = (lo + hi) // 2
                if key(a[mid]) < x:
                    lo = mid + 1
                else:
                    hi = mid
        return lo
else:
    from bisect import bisect_left
from typing import List, Tuple
from dataclasses import dataclass

# necessary to use bisect_left with ranges
@dataclass
class Interval:
    start: int
    end: int
    
    def __lt__(self, other) -> bool:
       return self.start < self.end < other.start
    
    def __eq__(self, other) -> bool:
       return self.start <= other.start <= self.end

def get_sentences_from_clusters(clusters: List[List[Tuple[int, int]]], sentences: List[Sentence]) -> List[List[int]]:
    '''find the sentence each cluster belongs to'''
    sentence_clusters = []
    for cluster in clusters:
        sentence_clusters.append([bisect_left(sentences, Interval(entity[0], entity[1]), key=lambda x: Interval(x.start, x.end)) for entity in cluster])

    return sentence_clusters

text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'
sentences = get_sentences_from_text(text)
clusters = [[(0, 5), (39, 42), (79, 82)]]
get_sentences_from_clusters(clusters, sentences)

[[0, 1, 1]]

In [9]:
from typing import List, Set
from itertools import groupby
from operator import itemgetter
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

def get_rule_groups_from_sentence_clusters(sentences: List[Sentence], sentence_clusters: List[List[int]]) -> List[List[int]]:
    def normalize_group(group: Set[int]) -> List[List[int]]:
        '''each group could contain multiple consecutive sublists. this method split these sublists'''
        res = []

        # https://stackoverflow.com/a/23861347/5587393
        for k, g in groupby(enumerate(sorted(list(group))), lambda x: x[0] - x[1]):
            res.append(list(map(itemgetter(1), g)))

        return res
    # the graph is built as a directed sparse graph where the first element of each cluster
    # is connected to the other elements in the same cluster
    graph = [[0 for _ in range(len(sentences))] for __ in range(len(sentences))]
    for cluster in sentence_clusters:
        for sentence in cluster[1:]:
            graph[cluster[0]][sentence] = 1

    # find the connected components of the graph created from the clusters returned after coref     
    graph = csr_matrix(graph)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    groups = [set() for _ in range(n_components)]
    for i, label in enumerate(labels):
        groups[label].add(i)

    return [norm_group for group in groups for norm_group in normalize_group(group)]

In [10]:
import itertools
from typing import List, Tuple
import pandas as pd

def get_rules(text: str) -> List[str]:
    text = clean_text(text)
    sentences = get_sentences_from_text(text)
    
    doc = nlp(text, component_cfg = { "fastcoref": {'resolve_text': True} })
    coref_clusters = doc._.coref_clusters
    logger.debug(coref_clusters)
    
    sentence_clusters = get_sentences_from_clusters(coref_clusters, sentences)
    rule_groups = get_rule_groups_from_sentence_clusters(sentences, sentence_clusters)
    
    return ['. '.join([sentences[s_index].content for s_index in group]) for group in rule_groups]

def get_rules_features(text: str) -> Tuple[int, float]:
    rules = get_rules(text)
    rule_count = len(rules)
    return rule_count, len(text) / rule_count

def remove_columns_prefix(df: pd.core.frame.DataFrame) -> None:
    '''remove prefix 'info.' from the columns of df'''
    df.rename(columns=lambda c: c.rsplit('.', 1)[-1], inplace=True)
    
df_features = pd.DataFrame()
with pd.read_csv(DATASET_FILE_PATH, chunksize=5) as reader:
    for df in reader:
        remove_columns_prefix(df)
        df_rules_features = df.apply(lambda x: pd.Series(get_rules_features(x.rulebook), 
                                     index=['rule_count', 'avg_rule_len']), axis='columns')
        df_features = pd.concat([df_features, df[['numweights', 'averageweight', 'playingtime', 'family']].join(df_rules_features)])
        
display(df_features)

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-07 23:07:50,180 bgg_predict  DEBUG    [[(200, 211), (287, 296), (929, 938), (1520, 1529)], [(439, 453), (463, 467), (530, 546)], [(632, 648), (649, 651)], [(714, 741), (754, 756)], [(689, 700), (777, 781), (840, 851)], [(785, 799), (819, 821)], [(917, 938), (986, 998)], [(1217, 1228), (1235, 1238), (1267, 1269)], [(392, 406), (1295, 1311)], [(1484, 1492), (1499, 1502), (1533, 1535)], [(468, 489), (1975, 1996)], [(2027, 2033), (2069, 2082), (2108, 2110)], [(2219, 2227), (2237, 2245), (2569, 2577), (2681, 2689), (2708, 2716)], [(2278, 2286), (2306, 2309), (2350, 2391)], [(2406, 2451), (2443, 2446)], [(2511, 2522), (2591, 2603), (2645, 2649)], [(2581, 2590), (2664, 2666)], [(2336, 2349), (2723, 2736)], [(2999, 3011), (3100, 3112)], [(2912, 2921), (3123, 3132)]]
DEBUG:bgg_predict:[[(200, 211), (287, 296), (929, 938), (1520, 1529)], [(439, 453), (463, 467), (530, 546)], [(632, 648), (649, 651)], [(714, 741), (754, 756)], [(689, 700), (777, 781), (840, 851)], [(785, 799), (819, 821)]

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

2022-11-07 23:10:30,668 bgg_predict  DEBUG    [[(6, 19), (508, 517)], [(444, 473), (608, 635)], [(652, 675), (704, 706)], [(681, 695), (765, 779)], [(734, 779), (801, 805)], [(839, 862), (928, 932)], [(1150, 1155), (1188, 1193)], [(1599, 1610), (1662, 1670), (1706, 1711), (1731, 1739)], [(1830, 1837), (1866, 1870)], [(1910, 1944), (1993, 2010)], [(2046, 2047), (2078, 2079)], [(2269, 2288), (2313, 2317)], [(2443, 2450), (2490, 2495)], [(2507, 2514), (2564, 2568)], [(2664, 2672), (2765, 2769), (2791, 2802)], [(2815, 2831), (2863, 2867)], [(2782, 2790), (2873, 2887)], [(2877, 2902), (2937, 2941)]]
DEBUG:bgg_predict:[[(6, 19), (508, 517)], [(444, 473), (608, 635)], [(652, 675), (704, 706)], [(681, 695), (765, 779)], [(734, 779), (801, 805)], [(839, 862), (928, 932)], [(1150, 1155), (1188, 1193)], [(1599, 1610), (1662, 1670), (1706, 1711), (1731, 1739)], [(1830, 1837), (1866, 1870)], [(1910, 1944), (1993, 2010)], [(2046, 2047), (2078, 2079)], [(2269, 2288), (2313, 2317)], [(2443, 2450), (24

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-07 23:10:33,701 bgg_predict  DEBUG    [[(88, 104), (292, 300)], [(28, 42), (365, 376), (626, 637), (665, 670), (708, 712), (799, 810), (1394, 1405), (1620, 1631)], [(391, 399), (446, 454), (459, 462), (484, 492)], [(419, 428), (523, 532), (883, 892), (1416, 1425)], [(550, 559), (563, 568)], [(588, 603), (615, 620)], [(253, 261), (789, 797)], [(1285, 1289), (1322, 1326), (1351, 1360)], [(1611, 1631), (1665, 1675)], [(1676, 1685), (1688, 1692)], [(1723, 1734), (1742, 1752), (1818, 1822)], [(1923, 1936), (1940, 1944)], [(2079, 2088), (2118, 2122), (2131, 2136)], [(2093, 2103), (2206, 2215), (2316, 2326)], [(2177, 2182), (2272, 2279), (2310, 2312), (2346, 2348)]]
DEBUG:bgg_predict:[[(88, 104), (292, 300)], [(28, 42), (365, 376), (626, 637), (665, 670), (708, 712), (799, 810), (1394, 1405), (1620, 1631)], [(391, 399), (446, 454), (459, 462), (484, 492)], [(419, 428), (523, 532), (883, 892), (1416, 1425)], [(550, 559), (563, 568)], [(588, 603), (615, 620)], [(253, 261), (789, 797)], 

,numweights,averageweight,playingtime,family,rule_count,avg_rule_len
0,703,2.1579,60,['familygames'],35.0,90.371429
1,62,3.1452,90,['strategygames'],438.0,166.388128
2,100,1.8100,90,['thematic'],94.0,210.553191
3,3030,1.4858,30,['familygames'],119.0,120.487395
4,439,2.7813,120,['strategygames'],27.0,137.259259
5,259,2.2162,60,"['strategygames', 'familygames']",43.0,137.651163
6,546,1.8718,45,['familygames'],37.0,98.378378
7,148,2.9392,240,['strategygames'],293.0,192.754266
8,576,1.1580,90,"['partygames', 'familygames']",50.0,64.540000
9,656,1.1265,30,"['partygames', 'familygames']",16.0,147.375000
